In [ ]:
!nvidia-smi

Thu Jun 16 04:21:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.9 MB/s 
     |████████████████████████████████| 596 kB 46.3 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 6.6 MB 50.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import transformers
transformers.__version__

'4.19.4'

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import matplotlib.pyplot as plt

In [ ]:
!gdown 1CVJUTAVf4jdvf4NNA_EYrBVVPWVEc2C9

Downloading...
From: https://drive.google.com/uc?id=1CVJUTAVf4jdvf4NNA_EYrBVVPWVEc2C9
To: /content/comment_data_all.csv
100% 48.2M/48.2M [00:00<00:00, 94.1MB/s]


In [ ]:
df = pd.read_csv('comment_data_all.csv', index_col=0)
df.drop(['month', 'page'], axis=1, inplace=True)
df

,comment,star
0,휘뚜루마뚜루 입고 다니기 좋습니다 로얄라이프제품들은 다 성공하는 듯!따숩고 예쁘기까...,5
1,어깨 깡패됩니다 기장감이나 털 재질 전체적으로 전부 맘에 들어요,5
2,바지 짱 편합니다ㅎㅎ 색깔도 너무 마음에 들고 핏도 좋네요,5
3,입기 편하고 후드 탈부착 돼서 여러 무드 연출하기도 쉽고 좋은데 가죽냄새랑 마감이 ...,4
4,유니폼 브릿지 발마칸 코트! 세일.기간중 저렴하게 겟겟! 최고 말모말모,5
...,...,...
344135,가방이 아주 포인트 그 자체라서 빤딱하고 너무 좋아요,5
344136,보들보둘하고 사이즈도 괜찮고 색도 이쁘게 잘나왔음,5
344137,생각보다 클 줄 알고 샀다가 조금 작았네요 그래도 좋아요,4
344138,생각보다 색상이 밝고 사이즈는 조금 작게 나온거 같아요.길이는 좀 길지만 수선안하고...,5


In [ ]:
five = df[df['star'] == 5]
four = df[df['star'] == 4]
three = df[df['star'] == 3]
two = df[df['star'] == 2]
one = df[df['star'] == 1]

In [ ]:
print(len(five))
print(len(four))
print(len(three))               
print(len(two))                
print(len(one))

298540
39242
5332
563
463


In [ ]:
pos = df[df['star'] == 5] #298,540
nag = df[df['star'] < 5] #45,600

In [ ]:
pos['label'] = 1
nag['label'] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
pos = pos.iloc[:45000] # 이 부분만 데이터에 맞게 조정하면 됩니다
nag = nag.iloc[:45000]

In [ ]:
pos_train = pos.sample(frac=0.8, random_state=3000)
pos_test = pos.drop(pos_train.index)

In [ ]:
nag_train = nag.sample(frac=0.8, random_state=3000)
nag_test = nag.drop(nag_train.index)

In [ ]:
train = pd.concat([pos_train, nag_train]).reset_index().drop(['index','star'], axis=1) # 72,000
test = pd.concat([pos_test, nag_test]).reset_index().drop(['index','star'], axis=1) # 18,000

In [ ]:
train

,comment,label
0,디자인도 맘에 들고 가격은 좀 있지만 넓고 만족스러워요,1
1,덩치가 커서 맞는 옷이 잘 없는데도 제가 입어도 크네요! 편하게 입기 너무 좋아요....,1
2,지금 당장 캠퍼스를 활보하고싶은 디자인입니다. 스프링이 약한거만 빼면 다좋습니다.,1
3,잘 안알려진 맛집 찾아버렸다 냄새난다는데 초반엔 심하고 입다보면 안나요 이쁨,1
4,생각했던 딥그레이 색상이 잘 나와서 마음에 듭니다. 생각보다는 기장이 길기는 한데 ...,1
...,...,...
71995,가격이 저렴하고 편하게 스타일링 하기 정말 좋습니다. 하지만 먼지나 보폴같은게 많...,0
71996,뒷부분이 너무 이뻐요 소재가 두꺼워서 손이 잘안갈듯,0
71997,제 신체 스펙에 적당한 사이즈의 오버핏이며 지금인 늦 겨울에 딱 입기 알맞은 두께감입니다,0
71998,기본템으로 무난무난 예쁜데 머리가 좀 납작해보여서 아쉬워요 그래도 싸게 주고 사서 ...,0


In [ ]:
test = test.dropna(how = 'any')

In [ ]:
print(len(test))

18000


In [ ]:
from transformers import AutoTokenizer, TFGPT2Model
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
max_seq_len = 128

In [ ]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, data_labels = [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):

        bos_token = [tokenizer.bos_token]
        eos_token = [tokenizer.eos_token]
        tokens = bos_token + tokenizer.tokenize(example) + eos_token
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return input_ids, data_labels

In [ ]:
train_X, train_y = convert_examples_to_features(train['comment'], train['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 72000/72000 [00:14<00:00, 5091.79it/s]


In [ ]:
test_X, test_y = convert_examples_to_features(test['comment'], test['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 18000/18000 [00:03<00:00, 4913.29it/s]


In [ ]:
# 최대 길이: 128
input_id = train_X[0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)

단어에 대한 정수 인코딩 : [    1 14253  7235 33301  8022 13647 34535 11732 12158 18203 13902 32490
  8084     1     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩의 길이 : 128
정수 인코딩 복원 : </s> 디자인도 맘에 들고 가격은 좀 있지만 넓고 만족스러워요</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [ ]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.5.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [ ]:
max_seq_len = 128

In [ ]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer])

In [ ]:
print(outputs)

TFBaseModelOutputWithPastAndCrossAttentions(last_hidden_state=<KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'tfgpt2_model')>, past_key_values=(<KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')

In [ ]:
print(outputs[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 128, 768), dtype=tf.float32, name=None), name='tfgpt2_model/transformer/Reshape_2:0', description="created by layer 'tfgpt2_model'")


In [ ]:
print(outputs[1])

(<KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by layer 'tfgpt2_model')>, <KerasTensor: shape=(2, None, 12, 128, 64) dtype=float32 (created by 

In [ ]:
print(outputs[0][:, -1])

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf.__operators__.getitem/strided_slice:0', description="created by layer 'tf.__operators__.getitem'")


In [ ]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.classifier = tf.keras.layers.Dense(1,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='sigmoid',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        cls_token = self.dropout(cls_token)
        prediction = self.classifier(cls_token)

        return prediction

In [ ]:
model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.5.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [ ]:
model.fit(train_X, train_y, epochs=2, batch_size=32, validation_split=0.2)

Epoch 1/2
1800/1800 [==============================] - 1741s 962ms/step - loss: 0.6307 - accuracy: 0.6707 - val_loss: 0.7213 - val_accuracy: 0.5227
Epoch 2/2
1800/1800 [==============================] - 1733s 963ms/step - loss: 0.5639 - accuracy: 0.7219 - val_loss: 1.0999 - val_accuracy: 0.3875


In [ ]:
results = model.evaluate(test_X, test_y, batch_size=1024)
print("test loss, test acc: ", results)

18/18 [==============================] - 185s 10s/step - loss: 0.6851 - accuracy: 0.6514
test loss, test acc:  [0.68514084815979, 0.6513888835906982]


In [ ]:
def sentiment_predict(new_sentence):

  bos_token = [tokenizer.bos_token]
  eos_token = [tokenizer.eos_token]
  tokens = bos_token + tokenizer.tokenize(new_sentence) + eos_token
  input_id = tokenizer.convert_tokens_to_ids(tokens)
  input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]
  input_id = np.array([input_id])
  score = model.predict(input_id)[0][0]

  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [ ]:
sentiment_predict('신발 색감이 미춌어요 이쁙다길래 얼마나 이쁠까했는데 너무 이쁜거같아요!! 착용감도 편하고 너무 잘 산거같아요')

93.61% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('팔 부분이 많이 크긴 한데 막상 입으면 오버핏으로 예쁘게 나오는 것 같네요 만족스러워요 후드집업 위에 입어도 예쁘고 아무대나 걸쳐도 예뻐요')

78.42% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('옷이 구려요')

54.79% 확률로 긍정 리뷰입니다.



In [ ]:
sentiment_predict('배송이 늦어요 사이즈도 잘 안맞고 옷이 금방 헤져요')

64.17% 확률로 부정 리뷰입니다.



In [ ]:
sentiment_predict('허리가 작다는 평 많이 봤는데 진짜 바지통에 비해 허리가 작은거 맞아요..')

62.53% 확률로 부정 리뷰입니다.



In [ ]:
sentiment_predict('기본템으로 무난무난 예쁜데 머리가 좀 납작해보여서 아쉬워요')

65.48% 확률로 부정 리뷰입니다.

